In [1]:
import glob
import os
import sys

try:
    sys.path.append(glob.glob('carla/PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import argparse
import math
# import gin
# gin.enter_interactive_mode()
import hydra
from omegaconf import DictConfig

import logging
from numpy import random
import time

from utils.weather import Weather
from utils.vehicle_manager import VehicleManager
from utils.pedestrian_manager import PedestrianManager
from utils.common import get_actor_blueprints

In [2]:


# @hydra.main(version_base=None, config_path="conf", config_name="config")
# def main(
#     host:str, port:int, number_of_walkers: int, 
#     filterw: str, generationw: str,  
#     seedw: int ):
def main(cfg: DictConfig):
    logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)
    
    client = carla.Client(cfg.host, cfg.port)
    client.set_timeout(10.0)

    vcfg = hydra.compose(config_name="vehicle_manager.yaml")
    print(vcfg)
    tm = VehicleManager(client=client, asynch=cfg.asynch, cfg=vcfg)

    pcfg = hydra.compose(config_name="pedestrian_manager.yaml")
    print(pcfg)
    pm = PedestrianManager(client=client, cfg=pcfg)
    try:
        world = client.get_world()

        tm.spawn_vehicles(number_of_vehicles=30)
        pm.spawn_walkers(number_of_walkers=pcfg.number_of_walkers)
        
        # Example of how to use Traffic Manager parameters
        # traffic_manager.global_percentage_speed_difference(-30.0)

        # Dynamic Weather
        speed_factor = 1.0
        update_freq = 0.5 / speed_factor


        weather = Weather(world.get_weather())

        elapsed_time = 0.0

        # tm.spawn_vehicles(number_of_vehicles=40)
        synchronous_master = world.get_settings().synchronous_mode
        while True:
            if not cfg.asynch and synchronous_master:
                world.tick()
            else:
                # world.wait_for_tick()
                timestamp = world.wait_for_tick(seconds=30.0).timestamp
                elapsed_time += timestamp.delta_seconds
                if elapsed_time > update_freq:
                    weather.tick(speed_factor * elapsed_time)

                    # traffic_manager.global_percentage_speed_difference(30.0 - (10 * (weather._storm.rain / 100)))
                    world.set_weather(weather.weather)
                    sys.stdout.write('\r' + str(weather) + 12 * ' ')
                    # tm.spawn_vehicles(number_of_vehicles=20)
                    sys.stdout.flush()
                    elapsed_time = 0.0

    finally:

        tm.destroy()
        pm.destroy()
        time.sleep(0.5)

In [3]:
# gin.parse_config_file('configs/config.gin')

try:
    with hydra.initialize(version_base=None, config_path="conf"):
        cfg = hydra.compose(config_name="config.yaml")
        print(cfg)
        main(cfg)
except KeyboardInterrupt:
    pass
finally:
    print('\ndone.')

{'host': '127.0.0.1', 'port': 2000, 'asynch': True}
{'host': '127.0.0.1', 'port': 2000, 'asynch': True, 'tm_port': 8000, 'safe': True, 'filterv': 'vehicle.*', 'generationv': 'All', 'hybrid': False, 'seed': 13, 'car_lights_on': False, 'hero': False, 'respawn': False, 'no_rendering': False}
You are currently in asynchronous mode. If this is a traffic simulation,             you could experience some issues. If it's not working correctly, switch to synchronous             mode by using self.traffic_manager.set_synchronous_mode(True)
{'host': '127.0.0.1', 'port': 2000, 'asynch': True, 'number_of_walkers': 10, 'filterw': 'walker.pedestrian.*', 'generationw': '2', 'seedw': 0, 'running_percentage': 0.0, 'crossing_percentage': 0.0}

Added 30 vehicles
Spawned 10 walkers.
Sun(alt: -12.24, azm: 320.15) Storm(clouds=8%, rain=0%, wind=5%)            
destroying 30 vehicles

Destroying 10 walkers

done.


In [25]:
speed_factor = 2.0
update_freq = 0.1 / speed_factor

client = carla.Client('127.0.0.1', 2000)
client.set_timeout(2.0)
world = client.get_world()

weather = Weather(world.get_weather())

elapsed_time = 0.0

while True:
    timestamp = world.wait_for_tick(seconds=30.0).timestamp
    elapsed_time += timestamp.delta_seconds
    if elapsed_time > update_freq:
        weather.tick(speed_factor * elapsed_time)
        world.set_weather(weather.weather)
        sys.stdout.write('\r' + str(weather) + 12 * ' ')
        sys.stdout.flush()
        elapsed_time = 0.0

Sun(alt: 46.79, azm: 15.15) Storm(clouds=0%, rain=0%, wind=5%)                 

KeyboardInterrupt: 

In [4]:
spawn_points = world.get_map().get_spawn_points()
number_of_spawn_points = len(spawn_points)
number_of_spawn_points

155

In [7]:
weather = world.get_weather()

In [21]:
weather.precipitation = 50
weather.wetness = 30
world.set_weather(weather)
world.get_weather().precipitation

50.0